<a href="https://colab.research.google.com/github/wilzamguerrero/SDZ/blob/main/SDZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/drive.png?raw=true' height="50" alt="netdata"/></center>
from IPython.display import clear_output
from google.colab import drive
drive.mount('/content/drive')
clear_output()
print('[1;32mDrive Connected ✓')



#markdown Download tool
#import os
#from IPython.display import clear_output

#save_models_dir = "/content/sample_data"#@param {type:"string"}
#os.chdir(save_models_dir)
#url = "https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt" #@param {type:"string"}
#!wget --no-check-certificate $url

#clear_output()
#print('[1;32mModel Save ✓')
     

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZanim2.gif?raw=true' height="50" alt="netdata"/></center> <a href="https://wilzamguerrero.ml/3c030fffd57c4e3bb4dd62e3971cb6a9"><font face='Verdana Regular' color='#80ff3' size='4'><left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/how_to_use5.png?raw=true' height="42" alt="netdata"/></center></font></a> <a href="https://zero01101.github.io/openOutpaint/"><font face='Verdana Regular' color='#80ff3' size='4'><left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/open_out_paint.png?raw=true' height="42" alt="netdata"/></center></font></a> <a href="https://www.painthua.com/"><font face='Verdana Regular' color='#80ff3' size='4'><left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/paint_hua.png?raw=true' height="42" alt="netdata"/></center></font></a> </a> <a href="https://github.com/Linaqruf/kohya-trainer"><font face='Verdana Regular' color='#80ff3' size='4'><left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/kohya2.png?raw=true' height="42" alt="netdata"/></center></font></a> </a> <a href="https://zhuyu1997.github.io/open-pose-editor/"><font face='Verdana Regular' color='#80ff3' size='4'><left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/save.png?raw=true' height="42" alt="netdata"/></center></font></a> </center></font></a>
##@markdown <left><font face='Squired Regular' color='#00f3f3' size='4'>Stable Diffusion Z</font>
##@markdown <left><font face='Verdana Regular' color='#00f3f3' size='4'>SDZboton</font> <a href="https://pbs.twimg.com/profile_images/1022127431447195649/ASyO_yY3_400x400.jpg"><font face='Verdana Regular' color='#80ff3' size='4'>SDZboton</font></a>
import os
import shutil
import sys
import fileinput
import time
from IPython.utils import capture
from IPython.display import clear_output, display, HTML
from datetime import timedelta
from google.colab import drive
from IPython.utils import capture
from subprocess import getoutput
from urllib.parse import unquote

try:
  start_colab
except:
  start_colab = int(time.time())-5
clear_output()
print("[1;90m") #Cyan text

try:
  output 
except:
  print('Checking GPU...', end='')
  output = getoutput('nvidia-smi --query-gpu=gpu_name --format=csv')
  if "name" in output:
    gpu_name = output[5:]
    print('\rCurrent GPU:', gpu_name, flush=True)
  else:
    print('\r\[1;90m ERROR: No GPU detected. Please do step below to enable.\n', flush=True)
    display(HTML("<img src='https://i.ibb.co/HC9KH17/NVIDIA-Share-23-01-02-173037.png' width='800px'/>"))
    time.sleep(5)
    from google.colab import runtime
    runtime.unassign()

#@markdown <left><font face='Verdana Regular' color='#00f3f3' size='4'>▣</font>
SDZ_install = "/content" 
SDZ_Version = "V1.5" #@param ["V1.5", "V2.1"]
SDZ_Download_Model = "V1.5.SPYBG_ToolKit5" #@param ["", "V1.5", "V1.5.inpaint", "V1.5.SPYBG_ToolKit5", "V1.5.NeverEndingDream12", "V1.5.Dreamlike_Diffusion_1", "V1.5.Deliberate_2", "V1.5.DreamShaper", "V1.5.CyriousMix", "V1.5.Protogenx58", "V1.5.RealisticVision2",  "V2.1", "V2.1.Perfect_World_3"]
SDZ = "SDZ"
SDZ_webui_path = (SDZ_install+'/'+'SDZ')
os.chdir(SDZ_install)
##@markdown <left><font face='Verdana Regular' color='#00f3f3' size='4'>▥</font>
#SDZ_Model_Init = "" #@param {type:"string"}
#SDZ_Models_Dir = "" #@param {type:"string"}
SDZ_Training = "" #@param ["", "New_Dreambooth - Kohya", "Old_Dreambooth"]
SDZ_Args = "--share --api --xformers --no-hashing --gradio-queue --theme=dark --cloudflared --disable-safe-unpickle --enable-insecure-extension-access --no-half-vae --opt-sdp-attention --disable-console-progressbars --cors-allow-origins=https://www.painthua.com,https://zero01101.github.io"
SDZ_Extensions = True
SDZ_ControlNet = True
SDZ_Big_Models = True
SDZ_Upscalers = True
#@markdown <left><font face='Verdana Regular' color='#00f3f3' size='4'>◨</font>
SDZ_Continue = False #@param {type:"boolean"}
SDZconfigV2="--config v2-inference-v.yaml"
#share = " --ckpt=" + SDZ_Model_Init + " --ckpt-dir=" + SDZ_Models_Dir + " " + SDZ_Args
share = SDZ_Args
stable_build = False

# SDZ_RunSDZ
if not SDZ_Continue: 
  with capture.capture_output() as cap:
    if os.path.exists(SDZ):
      shutil.rmtree(SDZ)
    !git clone --depth 1 https://github.com/AUTOMATIC1111/stable-diffusion-webui SDZ

    if not os.path.exists(SDZ):
      !git clone --depth 1 https://github.com/AUTOMATIC1111/stable-diffusion-webui SDZ

    %cd SDZ
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/SDZ_Files.txt
    import os; [os.system(line.strip()) for line in open(SDZ_webui_path+'/''SDZ_Files.txt')]; 

    clear_output()
  print('[1;96mInstalling SDZ ✓')

if SDZ_Continue:
  with capture.capture_output() as cap:
    os.chdir(SDZ_webui_path)
    clear_output()
  print('[1;96mPreparing SDZ Continue ✓')

# Unpack Repo, Dependencies, Caches
if not os.path.exists("/content/SDZ"):
  start_install = int(time.time())
  print("🚀 Unpacking... Please do not stop this process at all cost...", end='')
  with capture.capture_output() as cap:

    !wget https://huggingface.co/NoCrypt/fast-repo/resolve/main/ubuntu_deps.zip && unzip ubuntu_deps.zip -d ./deps && dpkg -i ./deps/* && rm -rf ubuntu_deps.zip /content/deps/

    if stable_build:
      !echo -e "https://huggingface.co/NoCrypt/fast-repo/resolve/90539f99b909f358ac3d8bc29a1be3e70db78a95/dep.tar.lz4\n\tout=dep.tar.lz4\nhttps://huggingface.co/NoCrypt/fast-repo/resolve/main/repo.tar.lz4\n\tout=repo.tar.lz4\nhttps://huggingface.co/NoCrypt/fast-repo/resolve/90539f99b909f358ac3d8bc29a1be3e70db78a95/cache.tar.lz4\n\tout=cache.tar.lz4\n" \
        | aria2c -i- -j5 -x16 -s16 -k1M -c 
    else:
      !echo -e "https://huggingface.co/NoCrypt/fast-repo/resolve/main/dep.tar.lz4\n\tout=dep.tar.lz4\nhttps://huggingface.co/NoCrypt/fast-repo/resolve/main/repo.tar.lz4\n\tout=repo.tar.lz4\nhttps://huggingface.co/NoCrypt/fast-repo/resolve/main/cache.tar.lz4\n\tout=cache.tar.lz4\n" \
        | aria2c -i- -j5 -x16 -s16 -k1M -c 
    
    !tar -xI lz4 -f dep.tar.lz4 --overwrite-dir --directory=/usr/local/lib/python3.10/dist-packages/ #(manual dir)
    !tar -xI lz4 -f repo.tar.lz4 --directory=/ #/content/sdw/ (auto dir)
    !tar -xI lz4 -f cache.tar.lz4 --directory=/ #/root/.cache/huggingface (auto dir)

    !rm -rf /content/dep.tar.lz4 /content/repo.tar.lz4 /content/cache.tar.lz4
    os.environ["SAFETENSORS_FAST_GPU"]='1'
    !apt -y update -qq
    !apt install -qq libunwind8-dev
    %env LD_PRELOAD=libtcmalloc.so
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = "garbage_collection_threshold:0.9,max_split_size_mb:512"
    !rm /content/SDZ/extensions/sd-webui-tunnels/id_rsa.pub /content/SDZ/extensions/sd-webui-tunnels/id_rsa
    del cap
  if not os.path.exists(hypernetworks_dir):
    os.makedirs(hypernetworks_dir)

  install_time = timedelta(seconds=time.time()-start_install)
  print("\r Finished unpacking. Took","%02d:%02d:%02d ⚡\n" % (install_time.seconds / 3600, (install_time.seconds / 60) % 60, install_time.seconds % 60), end='', flush=True)

  !echo -n {start_colab} > /content/SDZ/static/colabTimer.txt
  print(" Colab timer integration complete! You can see your colab time inside webui.")

# SDZ_Requirements
with capture.capture_output() as cap:
  os.chdir(SDZ_webui_path)
  import os
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

  !pip install svglib
  !sudo apt-get install libcairo2-dev
  !pip install -q torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U
  #pip install -q xformers==0.0.18 triton==2.0.0 -U
  #!pip install gradio

  !wget -N https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml

# SDZ_Models
  os.chdir(SDZ_webui_path)
  %cd models/Stable-diffusion
  if SDZ_Download_Model == "V1.5":
    !apt-get install -qq -y aria2 && url="https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt" && base_name=$(basename $url) && ((echo $url | grep -q "drive.google.com") && gdown --fuzzy $url || ((echo $url | grep -q "huggingface.co/") && (hf_token="" && aria2c --console-log-level=error --summary-interval=10 --header="Authorization: Bearer $hf_token" -c -x 16 -k 1M -s 16 $base_name $url)) || aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -o v15.ckpt $url)
    
  elif SDZ_Download_Model == "V1.5.inpaint":
    !apt-get install -qq -y aria2 && url="https://huggingface.co/uwg/modelz_base/resolve/main/v1-5-inpaint-fp16.ckpt" && base_name=$(basename $url) && ((echo $url | grep -q "drive.google.com") && gdown --fuzzy $url || ((echo $url | grep -q "huggingface.co/") && (hf_token="" && aria2c --console-log-level=error --summary-interval=10 --header="Authorization: Bearer $hf_token" -c -x 16 -k 1M -s 16 $base_name $url)) || aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -o v15inp.ckpt $url)

  elif SDZ_Download_Model == "V1.5.SPYBG_ToolKit5":
    !apt-get install -qq -y aria2 && url="https://civitai.com/api/download/models/17292" && base_name=$(basename $url) && ((echo $url | grep -q "drive.google.com") && gdown --fuzzy $url || ((echo $url | grep -q "huggingface.co/") && (hf_token="" && aria2c --console-log-level=error --summary-interval=10 --header="Authorization: Bearer $hf_token" -c -x 16 -k 1M -s 16 $base_name $url)) || aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -o 15SPYBG_ToolKit5.safetensors $url)

  elif SDZ_Download_Model == "V1.5.NeverEndingDream12":
    !apt-get install -qq -y aria2 && url="https://civitai.com/api/download/models/64094" && base_name=$(basename $url) && ((echo $url | grep -q "drive.google.com") && gdown --fuzzy $url || ((echo $url | grep -q "huggingface.co/") && (hf_token="" && aria2c --console-log-level=error --summary-interval=10 --header="Authorization: Bearer $hf_token" -c -x 16 -k 1M -s 16 $base_name $url)) || aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -o 15NeverEndingDream12.safetensors $url)

  elif SDZ_Download_Model == "V1.5.Dreamlike_Diffusion_1":
    !apt-get install -qq -y aria2 && url="https://civitai.com/api/download/models/1356" && base_name=$(basename $url) && ((echo $url | grep -q "drive.google.com") && gdown --fuzzy $url || ((echo $url | grep -q "huggingface.co/") && (hf_token="" && aria2c --console-log-level=error --summary-interval=10 --header="Authorization: Bearer $hf_token" -c -x 16 -k 1M -s 16 $base_name $url)) || aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -o 15Dreamlike_Diffusion_1.ckpt $url)

  elif SDZ_Download_Model == "V1.5.Deliberate_2":
    !apt-get install -qq -y aria2 && url="https://civitai.com/api/download/models/15236" && base_name=$(basename $url) && ((echo $url | grep -q "drive.google.com") && gdown --fuzzy $url || ((echo $url | grep -q "huggingface.co/") && (hf_token="" && aria2c --console-log-level=error --summary-interval=10 --header="Authorization: Bearer $hf_token" -c -x 16 -k 1M -s 16 $base_name $url)) || aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -o 15Deliberate_2.safetensors $url)

  elif SDZ_Download_Model == "V1.5.DreamShaper":
    !apt-get install -qq -y aria2 && url="https://civitai.com/api/download/models/43888" && base_name=$(basename $url) && ((echo $url | grep -q "drive.google.com") && gdown --fuzzy $url || ((echo $url | grep -q "huggingface.co/") && (hf_token="" && aria2c --console-log-level=error --summary-interval=10 --header="Authorization: Bearer $hf_token" -c -x 16 -k 1M -s 16 $base_name $url)) || aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -o 15DreamShaper.safetensors $url)

  elif SDZ_Download_Model == "V1.5.CyriousMix":
    !apt-get install -qq -y aria2 && url="https://civitai.com/api/download/models/7951" && base_name=$(basename $url) && ((echo $url | grep -q "drive.google.com") && gdown --fuzzy $url || ((echo $url | grep -q "huggingface.co/") && (hf_token="" && aria2c --console-log-level=error --summary-interval=10 --header="Authorization: Bearer $hf_token" -c -x 16 -k 1M -s 16 $base_name $url)) || aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -o 15CyriousMix.safetensors $url)

  elif SDZ_Download_Model == "V1.5.Protogenx58":
    !apt-get install -qq -y aria2 && url="https://huggingface.co/darkstorm2150/Protogen_x5.8_Official_Release/resolve/main/ProtoGen_X5.8-pruned-fp16.safetensors" && base_name=$(basename $url) && ((echo $url | grep -q "drive.google.com") && gdown --fuzzy $url || ((echo $url | grep -q "huggingface.co/") && (hf_token="" && aria2c --console-log-level=error --summary-interval=10 --header="Authorization: Bearer $hf_token" -c -x 16 -k 1M -s 16 $base_name $url)) || aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -o 15Protogenx58.safetensors $url)

  elif SDZ_Download_Model == "V1.5.RealisticVision2":
    !apt-get install -qq -y aria2 && url="https://civitai.com/api/download/models/29460" && base_name=$(basename $url) && ((echo $url | grep -q "drive.google.com") && gdown --fuzzy $url || ((echo $url | grep -q "huggingface.co/") && (hf_token="" && aria2c --console-log-level=error --summary-interval=10 --header="Authorization: Bearer $hf_token" -c -x 16 -k 1M -s 16 $base_name $url)) || aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -o 15RealisticVision2.safetensors $url)

  elif SDZ_Download_Model == "V2.1":
    !apt-get install -qq -y aria2 && url="https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt" && base_name=$(basename $url) && ((echo $url | grep -q "drive.google.com") && gdown --fuzzy $url || ((echo $url | grep -q "huggingface.co/") && (hf_token="" && aria2c --console-log-level=error --summary-interval=10 --header="Authorization: Bearer $hf_token" -c -x 16 -k 1M -s 16 $base_name $url)) || aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -o v21.ckpt $url)

  elif SDZ_Download_Model == "V2.1.Perfect_World_V3":
    !apt-get install -qq -y aria2 && url="https://civitai.com/api/download/models/65269" && base_name=$(basename $url) && ((echo $url | grep -q "drive.google.com") && gdown --fuzzy $url || ((echo $url | grep -q "huggingface.co/") && (hf_token="" && aria2c --console-log-level=error --summary-interval=10 --header="Authorization: Bearer $hf_token" -c -x 16 -k 1M -s 16 $base_name $url)) || aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -o 21Perfect_World_3.safetensors $url)

  else:
    SDZ_Download_Model == ""
    os.chdir(SDZ_webui_path)

# SDZ_Extensions
  if SDZ_Extensions:
    os.chdir(SDZ_webui_path)
    %cd models/VAE
    !wget --timestamping https://huggingface.co/uwg/modelz_base/resolve/main/VAE/vae-ft-mse-840000-ema-pruned.ckpt
    %cd ..

    if not os.path.exists('Lora'):
        os.makedirs('Lora')
    %cd Lora
    !wget --timestamping https://huggingface.co/uwg/modelz_base/resolve/main/Lora/epiNoiseoffset_v2.safetensors
    !wget --timestamping https://huggingface.co/uwg/modelz_base/resolve/main/Lora/theovercomer8sContrastFix_sd15.safetensors
    !wget --timestamping https://huggingface.co/uwg/modelz_base/resolve/main/Lora/to8sHighKeyLORASD15SD2_sd15.safetensors
    %cd ../..

    %cd embeddings
    !wget --timestamping https://huggingface.co/uwg/modelz_base/resolve/main/embeddings/bad-hands-5.pt
    !wget --timestamping https://huggingface.co/uwg/modelz_base/resolve/main/embeddings/easynegative.safetensors
    %cd ..

    %cd extensions
    !git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111.git
    !git clone https://github.com/camenduru/sd-civitai-browser.git
    !git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git
    !git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser.git
    !git clone https://github.com/etherealxx/batchlinks-webui.git
    !git clone https://github.com/camenduru/sd-webui-tunnels
    #!git clone https://github.com/volotat/SD-CN-Animation.git
    #!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-promptgen.git
    #!git clone https://github.com/imrayya/stable-diffusion-webui-Prompt_Generator.git
    #!git clone https://github.com/canisminor1990/sd-web-ui-kitchen-theme.git
    #!git clone https://github.com/v8hid/infinite-zoom-automatic1111-webui.git
    #!git clone https://github.com/Extraltodeus/depthmap2mask.git
    #!git clone https://github.com/fkunn1326/openpose-editor.git
    #!git clone https://github.com/opparco/stable-diffusion-webui-two-shot.git
    #!git clone https://github.com/opparco/stable-diffusion-webui-composable-lora.git
    #!git clone https://github.com/jexom/sd-webui-depth-lib.git
    #!git clone https://github.com/alemelis/sd-webui-ar.git
    #!git clone https://github.com/hnmr293/posex.git
    #!git clone https://github.com/nonnonstop/sd-webui-3d-open-pose-editor.git
    #!git clone https://github.com/butaixianran/Stable-Diffusion-Webui-Civitai-Helper.git
    #!git clone https://github.com/Scholar01/sd-webui-mov2mov.git
    #!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-pixelization.git
    #!git clone https://github.com/s9roll7/ebsynth_utility.git
    #!git clone https://github.com/journey-ad/sd-webui-bilingual-localization.git
    #!git clone https://github.com/hako-mikan/sd-webui-supermerger.git
    #!git clone https://github.com/zero01101/openOutpaint-webUI-extension.git
    #!git clone https://github.com/Bing-su/sd-webui-tunnels.git
    #!git clone https://github.com/bbc-mc/sdweb-merge-block-weighted-gui.git
    #!git clone https://github.com/bbc-mc/sdweb-xyplus.git
    #!git clone https://github.com/KohakuBlueleaf/a1111-sd-webui-locon.git
    #!git clone https://github.com/arenatemp/stable-diffusion-webui-model-toolkit.git
    #!git clone https://github.com/fishslot/video_loopback_for_webui.git
    #!git clone https://github.com/mcmonkeyprojects/sd-dynamic-thresholding.git
    #!git clone https://github.com/kohya-ss/sd-webui-additional-networks.git
    #!git clone https://github.com/hnmr293/sd-webui-llul.git
    #!git clone https://github.com/hnmr293/sd-webui-cutoff.git
    #!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg
    #!git clone https://github.com/camenduru/stable-diffusion-webui-huggingface
    #!git clone https://github.com/AIrjen/OneButtonPrompt.git
    !git clone https://github.com/Physton/sd-webui-prompt-all-in-one.git
    !git clone https://github.com/deforum-art/deforum-for-automatic1111-webui.git
    #!git clone https://github.com/zanllp/sd-webui-infinite-image-browsing.git
    #!git clone https://github.com/Uminosachi/sd-webui-inpaint-anything.git
    #!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py
    #!git clone https://github.com/CiaraStrawberry/TemporalKit.git
    %cd ..

# SDZ_ControlNet
  if SDZ_ControlNet:
    os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Mikubill/sd-webui-controlnet.git
    #!git clone https://github.com/crucible-ai/sd-webui-controlnet.git
    
    %cd sd-webui-controlnet/models
    import os; from concurrent.futures import ThreadPoolExecutor; [executor.submit(os.system, line.strip()) for line in open(SDZ_webui_path+'/SDZ_ControlNet.txt') if not os.path.exists('t2iadapter_color-fp16.safetensors') for executor in [ThreadPoolExecutor(max_workers=len(open(SDZ_webui_path+'/SDZ_ControlNet.txt').readlines()))]];
    !cp cldm_v15.yaml cldm_v21.yaml {os.path.join(SDZ_webui_path, 'models')}
    %cd ../../..

# SDZ_Upscalers
  if SDZ_Upscalers:
    os.chdir(SDZ_webui_path+'/'+'models')
    if not os.path.exists('ESRGAN'):
        os.makedirs('ESRGAN')
    %cd ESRGAN
    import os; from concurrent.futures import ThreadPoolExecutor; [executor.submit(os.system, line.strip()) for line in open(SDZ_webui_path+'/SDZ_Upscalers.txt') if not os.path.exists('4x-UltraSharp.pth') for executor in [ThreadPoolExecutor(max_workers=len(open(SDZ_webui_path+'/SDZ_Upscalers.txt').readlines()))]];

# SDZ_Training
  if SDZ_Training == "New_Dreambooth - Kohya":
    os.chdir(SDZ_webui_path)
    if not os.path.exists('/usr/local/lib/python3.10/site-packages/open_clip'):
      !wget -N https://github.com/korakot/kora/releases/download/v0.10/py310.sh
      !bash ./py310.sh -b -f -p /usr/local
      !python -m ipykernel install --name "py310" --user
      !pip install open-clip-torch
      !pip install triton
      !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
    %cd extensions
    !git clone https://github.com/ddpn08/kohya-sd-scripts-webui.git
    !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
    %cd ..
    !pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 -f https://download.pytorch.org/whl/cu117/torch_stable.html

  elif SDZ_Training == "Old_Dreambooth":
    os.chdir(SDZ_webui_path)
    if not os.path.exists('/usr/local/lib/python3.10/site-packages/open_clip'):
      !wget -N https://github.com/korakot/kora/releases/download/v0.10/py310.sh
      !bash ./py310.sh -b -f -p /usr/local
      !python -m ipykernel install --name "py310" --user
      !pip install open-clip-torch
      !pip install triton
      !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
    
    os.chdir(SDZ_install)
    %cd stable-diffusion-webui
    !git checkout 1130d5df669911a5c67696be90bccca3ecf5f487
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user_old.css -O user.css
    %cd extensions
    !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
    %cd sd_dreambooth_extension
    !git checkout f9900ccb3ce7025d1c89334e320a1c3b255bbd07
    %cd ../..

  else:
    SDZ_Training == ""
    os.chdir(SDZ_webui_path)

  if SDZ_Big_Models:
    #!sed -i -e 's/\"sd_model_checkpoint\"\,/\"sd_model_checkpoint\,sd_vae\,CLIP_stop_at_last_layers\"\,/g' $SDZ_webui_path'/modules/shared.py'
    #!sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" $SDZ_webui_path'/modules/sd_models.py'
    !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' $SDZ_webui_path'/webui.py'
    !sed -i "s@map_location='cpu'@map_location='cuda'@" $SDZ_webui_path'/modules/extras.py'
    #!sed -i '902s@.*@        self.logvar = self.logvar.to(self.device)@' $SDZ_webui_path'/repositories/stable-diffusion-stability-ai/ldm/models/diffusion/ddpm.py'
    #!sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda' if os.path.getsize(checkpoint_file) > 5500000000 else map_location@" $SDZ_webui_path'/modules/sd_models.py'
    #!sed -i "s@'cpu'@'cuda'@" $SDZ_webui_path'/modules/extras.py'
    !sed -i "s/                    print(strings.en\[\"SHARE_LINK_MESSAGE\"\])/                    print('[1;90m')/" '/usr/local/lib/python3.9/dist-packages/gradio/blocks.py'

    clear_output()
print('[1;90mPreparing SDZ System ✓')
if SDZ_Version == "V1.5":
    !COMMANDLINE_ARGS="{SDZ_Args}" REQS_FILE="requirements_versions.txt" python launch.py 

elif SDZ_Version == "V2.1":
    !COMMANDLINE_ARGS="{SDZ_Args}" REQS_FILE="requirements_versions.txt" python launch.py SDZconfigV2

time_since_start = timedelta(seconds=time.time()-start_colab)
print("\n\n[1;96mYou've been running this colab for","%02d:%02d:%02d" % (time_since_start.seconds / 3600, (time_since_start.seconds / 60) % 60, time_since_start.seconds % 60))
print("\n\n")


